In [ ]:
%%capture
!pip install opendatasets
!pip install imutils
!pip install facenet_pytorch

In [ ]:
import opendatasets as od
import zipfile, urllib.request, shutil, requests
import numpy as np
import pandas as pd
import os
from pathlib import Path
import IPython
import cv2
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import matplotlib.image as image
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()
import glob
from PIL import Image
from imutils.video import FileVideoStream
import time
import torch
from facenet_pytorch import MTCNN
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.mkdir('/content/data')
os.mkdir('/content/train')

In [ ]:
#links of the data parts to be downlaoded
parts = [
    'https://storage.googleapis.com/kaggle-competitions-detached-data/16880/dfdc_train_part_24.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1679572683&Signature=hBvVDvt7gGK26Hc7gDYkqYmw1LPM3QA7SyP%2B04MZmLxGMYqmHJTVm07Mi23um4uyX55PO0zHuMGPUkSiKrWFs4q6ZwHhjD7YTB%2FdKQiVnBqen4NvB58LLaDzQaEzirzLvIMVqWiL9T6dczDeerFe4f7c%2BNiFzAOei00aFOkzwrFB3kfoQH9eDkdSNZsptpLvkY0xWgoWUJce4MvNORIBXOS%2BqyVIK3gaUtK%2FGp6TDftYaU37mwKHvP4loCFsiR%2BHOGd2TQYDNcVZG9dk5ieuaY0iiJ%2F1VE9nvIyIOpj0xmozlOahxZsf1TqQJ0b1uhENgYQ6vxldbCOjqKbN8PPU%2BA%3D%3D',
    'https://storage.googleapis.com/kaggle-competitions-detached-data/16880/dfdc_train_part_25.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1679572703&Signature=smJv90POP9VjdLqh1xb%2B9YQ9c7yyzml%2BqkeLCLQE%2FyhYywSuSdfVsZYKXL6enr%2FICPSXa586%2BH%2FKbBE%2FqUR2SwzhFy5pMmKhZFEijuID6AdbqIv%2FtEmQqOkDMjH1RBSsZjoJBaEuaGGKEPwmxFycmWdBv8i2gKLb9sugzCeE1ti2Gm0kp3p2%2BfNUWKBGyHmY71rXL52Ii0HyCcYB9VJz8pajd1%2F0YcjriYOKx1l9xHm4cAYWfvas8loaEl4UWbaiVi1D011IP%2F%2BgwkE5y2WWgzs2Jnkn5exHb5EMsxXMjPrzFVXgnjmTbaUqTAR7wjCUNoxK%2FeBX9X0uUCgA%2FxCuaA%3D%3D',
    'https://storage.googleapis.com/kaggle-competitions-detached-data/16880/dfdc_train_part_26.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1679572723&Signature=FU%2Frcdy6rbHy9C6ruwrXciFYSR%2FJMScQ0Ya0yD20xxNkN%2Fc41OUo5oQnkrYYpgTT2WoQpaQVEQKB7MMfX2B3uZLCghLFcIXjdI3evyqHvN1eAziDcFJ4zUAYYXKHJ4sVC869bcsQJxk5nK5rLzPrH%2FCuXaYCpDmrs6ednhqR%2BEcqqreok6ju7CsS6PgeDekKhkP1mDy1Sbgl9aYOJNFyL4nzNuBIPWJzFjkzk4HQBPM4PwUGzfwY3l1Yg0mlDvoTIMur1z%2FzwVeXZ74CqGNnouf7uFn5VLel3ryvc6TqyAuciERttDj833MTyHWh9idciO7ucZJK8ZFUIiUW0dj%2BGA%3D%3D',
    'https://storage.googleapis.com/kaggle-competitions-detached-data/16880/dfdc_train_part_27.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1679572740&Signature=d%2BL5QLY5XonYPi7khIfaPKfnlM0azTxSDdRbbF24b49QBxsmzX%2B8Z8pTPDCQhkpTH8JAHTkxa7yNRFJXXAUYfs0MTxmLpbj03nb1t8WW6KORKTJGCDK%2Bq3UVfMRSjdcLXB6J2jB2kL%2BV4pYuVpDFZF22M2obm0%2BZ%2BqEVvxy1L7zR6vMJAo5heVKlaDoJU0juXSmd3qAxGyatywDs50b4wPyHQWAUoZ%2BjXaIBogsiNqxToxdxqBcUZI4FLhYSmVO6JXSAiTmEBa6aFy6e0WfKKKsbB9Jd02EJHlpgwZ1KJJcenTQvYeX7SN2UQ%2BSkEGEYr5yc92uqVu44wGRFC47mOA%3D%3D',
    'https://storage.googleapis.com/kaggle-competitions-detached-data/16880/dfdc_train_part_28.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1679572762&Signature=qojpGGfqiQYcISE4PeuGgRrI88uFrhwawOsB5oeEnfDuOBbzLIjrHiO1v4Qg2JKkr96cUmEBCcOd31NYyo7GtsAcmh1ONRVGvWU7jk3tRNX0gmyQY2zCpY8yoX%2BvKdR90fuhYmm0phqDHrvtm2ImyVvBMYXufRrXVtUburmEbRqEYF7l154HjqsOVBg51hqUyepJUImytyU%2BUfz%2BZ0F0lv%2BtPOypUAxJ5Iw4%2FOWL6c7mRfa2CTXtrcQcotXXDvoNotW86P7hlov7ZBmuWN8L6SYUfE7mcnplQmqJIADbnxi6W4xipFc5af6Nckwf24WcCMAzvyG24ddj81SYbyXJ5Q%3D%3D',
    'https://storage.googleapis.com/kaggle-competitions-detached-data/16880/dfdc_train_part_29.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1679572785&Signature=hM%2FuhR6Gs6dSFvRzNPLUZbbqS0LELgy8fKbyBseXs5LMOOvU0M8skOmgrjS0BkrzuFwhUiL%2Fsk1YIBZtCA3mcKZrVIEL8TzwuG6ERf6IRk02QQTbB1HLueZjhBiRqT0rQwe9svaa9wW2v6TDMNiizZOpfjrkTtWlTqcBuZREMF2NF5xT5hpfOAp1Rzx57Itrv0%2FNhP7ftMefDphJKki0XwjK%2B03IqCdH%2BKtcS0Juy6rScNYWPadyap4%2BLXoTx%2FblPwMvu%2F4YrIc8LyA8IhR6PIvlGdB74UVoOsyJf3vTs8IVWRaNE4Va9Im1rFEaUevK3zfaiE%2FouWcV1Upk1G6yfQ%3D%3D',
    'https://storage.googleapis.com/kaggle-competitions-detached-data/16880/dfdc_train_part_30.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1679572801&Signature=krzQjzepFQwSi25oH4jSJLXx%2BEuh5EPFRR0W63ZHliYuriN%2BBJ6KUat1VRL0uknuu2vyNXUFt469AwYXoAsP8rWBmSALHrgNsM8Nwh8cvg2%2F3OX9Ziv7U9FBJVvG2UusXkHQ57Kj7XiODRfLRZiIEWfXFJof86%2B%2BGfovFIEZEa5AjfdbHPYAV9ftj8nnp1ZgQaZHJsGPTEPK9jfZ3vdR9YZ%2FVkrxbHPCeEmXWszvHAVbpwcxr3G%2FgLYBFbSEIPDmwvjYzAE2NRejTxECGGRnhuGnxHwtro8L4A0QvzKmMR42gnwUuqx9t4HVqMk4USNeDp2oDpLJLPTWzjkcbcHNgA%3D%3D',
    'https://storage.googleapis.com/kaggle-competitions-detached-data/16880/dfdc_train_part_31.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1679572819&Signature=an1SNNy%2FeRYDWkZkXwxRutqZ944Z%2FEFMOBV%2BOYlMrsXmew1GBbzkcAQ4EoQBIec%2FDNfIfWbAa5bUpbB4e77%2BUSxLHXrHZ3kX5fsA%2FLuCF9goQfWkl%2FSNIJKdmtfHSbmuR53bO8LLKaKrX1VlHbjjpkc25W80Cho4az9MNKvM5DAfKOtTh3qjIHAPOWp3Yab7PdfyMqm21qldwoOvR4%2BywLxYVIF6CnwKSONyBNelmMy9ws7aj4n5dgSlmJNIajFP5nsn%2Bh2zsOOKvtTfoAYVE%2F6nQsW0WATE0DV7rnSO1kPkiVgavueIOUo3qoz3gZkVNy%2BqMJfSPRXPd9%2FiPeQ%2FfQ%3D%3D'
    ]

In [ ]:
#Donwload each part and save it in the files
for ind , part in enumerate(parts):
    response = requests.get(part, stream=True)
    with open(f'{ind}.zip', "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
#Extract the files from the downloaded parts then delet each part after extracing to save storage
for ind in range(len(parts)):
    f = f"/content/{ind}.zip"
    with zipfile.ZipFile(f, 'r') as zip_ref:
        zip_ref.extractall("/content/data")
    os.remove(f)

In [ ]:
class FastMTCNN(object):
    """Fast MTCNN implementation."""

    def __init__(self, stride, resize=1, *args, **kwargs):
        """Constructor for FastMTCNN class.

        Arguments:
            stride (int): The detection stride. Faces will be detected every `stride` frames
                and remembered for `stride-1` frames.

        Keyword arguments:
            resize (float): Fractional frame scaling. [default: {1}]
            *args: Arguments to pass to the MTCNN constructor. See help(MTCNN).
            **kwargs: Keyword arguments to pass to the MTCNN constructor. See help(MTCNN).
        """
        self.stride = stride
        self.resize = resize
        self.mtcnn = MTCNN(*args, **kwargs)

    def __call__(self, frames):
        """Detect faces in frames using strided MTCNN."""
        if self.resize != 1:
            frames = [
                cv2.resize(f, (int(f.shape[1] * self.resize), int(f.shape[0] * self.resize)))
                    for f in frames
            ]

        boxes, probs = self.mtcnn.detect(frames[::self.stride])
        faces = []
        for i, frame in enumerate(frames):
            box_ind = int(i / self.stride)
            if boxes[box_ind] is None:
                continue
            for box in boxes[box_ind]:
                box = [int(b) for b in box]
                faces.append(frame[box[1]:box[3], box[0]:box[2]])
        return faces

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
fast_mtcnn = FastMTCNN(
stride=4,
resize=0.5,
margin=50,
factor=0.6,
keep_all=True,
device=device,
min_face_size = 60
)

In [ ]:
def save_vid(frame_array,filename):
    if len(frame_array) < 60:
        pass
    frames = 0
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    out = cv2.VideoWriter('/content/train/'+os.path.basename(filename),fourcc, 60, (240,240))
    for frame in frame_array:
        try:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            frame = cv2.resize(frame,(240,240))
            out.write(frame)
            frames +=1
        except:
            pass
        if frames == 60:
            out.release()
            break

In [ ]:
def run_detection(fast_mtcnn, filenames):
    frames = []
    frames_processed = 0
    faces_detected = 0
    batch_size = 60
    start = time.time()
    for filename in tqdm(filenames):
        v_cap = FileVideoStream(filename).start()
       # print(filename)
        #print(v_cap.read().shape)
        v_len = int(v_cap.stream.get(cv2.CAP_PROP_FRAME_COUNT))
        for j in range(min(60,v_len)):
            frame = v_cap.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
            if len(frames) >= batch_size or j == v_len - 1:
                faces = fast_mtcnn(frames)
                frames_processed += len(frames)
                faces_detected += len(faces)
                frames = []
                print(
                    f'Frames per second: {frames_processed / (time.time() - start):.3f},',
                    f'faces detected: {faces_detected}\r',
                    end=''
                )
                save_vid(faces,filename)

        v_cap.stop()

In [ ]:
f = glob.glob('/content/data/*/*')

In [ ]:
len(f)

10122

In [ ]:
run_detection(fast_mtcnn,f)

  0%|          | 0/10122 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/facenet_pytorch/models/utils/detect_face.py:183: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  batch_boxes, batch_points = np.array(batch_boxes), np.array(batch_points)
/usr/local/lib/python3.9/dist-packages/facenet_pytorch/models/mtcnn.py:339: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  boxes = np.array(boxes)
/usr/local/lib/python3.9/dist-packages/facenet_pytorch/models/mtcnn.py:341: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or nd

/usr/local/lib/python3.9/dist-packages/facenet_pytorch/models/mtcnn.py:340: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  probs = np.array(probs)


In [ ]:
shutil.make_archive('/content/cropped0', 'zip', '/content/train/')

In [ ]:
shutil.copy("/content/cropped0.zip", "/content/drive/MyDrive")